In [6]:
import pulp
import itertools

In [7]:
def initialise_deterministic_minimisation_problem(
    specialties, hospitals, bands
):
    """
    Initialise the mininmisation problem. 
    Set decision variables for the models.
    """
    sh = [(s,h) for s in specialties for h in hospitals]
    shb = [(s,h,b) for s in specialties for h in hospitals for b in bands]

    prob = pulp.LpProblem("Deterministic", pulp.LpMinimize)
    
    xbed = pulp.LpVariable.dicts(
        "Xbed", (specialties, hospitals), lowBound=0, cat='Integer'
    )
    xstaff = pulp.LpVariable.dicts(
        "Xstaff", (specialties, hospitals, bands), lowBound=0,cat='Integer'
    )
    return prob, sh, shb, xbed, xstaff

def add_deterministic_constraints(xbed, xstaff, UBbed, UBstaff, D, K, R, sh, shb, prob):
    """
    Add the constaints that are required for the deterministic model
        
    - Constraints 1-6: Ensures demand is met across all specialties and all regions
    - Constraint 7: Ensures beds are only able to open in a ward if the facilities are able to be opened 
    - Constraint 8: Ensures staffing ratios are met
    - Constraint 9: Ensures beds deployed does not exceed maximum capacity of hospital
    - Constraint 10: Ensures staff deployed does not exceed maximum staffing resources
    """
    
    for s in specialties:
        prob += pulp.lpSum(xbed[s][h] for h in region1) >= pulp.lpSum(D[s][0]) #Constraint 1
        prob += pulp.lpSum(xbed[s][h] for h in region2) >= pulp.lpSum(D[s][1]) #Constraint 2
        prob += pulp.lpSum(xbed[s][h] for h in region3) >= pulp.lpSum(D[s][2]) #Constraint 3
        prob += pulp.lpSum(xbed[s][h] for h in region4) >= pulp.lpSum(D[s][3]) #Constraint 4
        prob += pulp.lpSum(xbed[s][h] for h in region5) >= pulp.lpSum(D[s][4]) #Constraint 5
        prob += pulp.lpSum(xbed[s][h] for h in region6) >= pulp.lpSum(D[s][5]) #Constraint 6
        
        for h in hospitals:
            prob += pulp.lpSum(xbed[s][h])<= pulp.lpSum(K[s][h]) # Constraint 7
                
            for b in bands:
                prob += pulp.lpSum(xstaff[s][h][b]) >= pulp.lpSum(R[s][b]*(xbed[s][h])) #Constraint 8
            
    for h in hospitals:
        prob += pulp.lpSum(xbed[s][h] for s in specialties) <= UBbed[h]  #Constraint 9
        
    for b in bands:
        prob += pulp.lpSum(xstaff[s][h][b] for (s,h) in sh) <= UBstaff[b] # #Constraint 10
        
    return prob

def solve_deterministic_minimisation_problem(
    specialties,
    bands,
    region1,
    region2,
    region3,
    region4,
    region5,
    region6,
    hospitals,
    regions,
    D,
    K,
    R,
    cbed,
    cstaff,
    UBbed,
    UBstaff,
):
    """
    Solves the deterministic problem, with the objective function being minimised
    Uses the CBC_CMD Solver, with maximum run time of 60 seconds and precision of 0.01
    """
    prob, sh, shb, xbed, xstaff = initialise_deterministic_minimisation_problem(
        specialties=specialties, 
        hospitals=hospitals, 
        bands=bands
    )
    
    prob += (
        pulp.lpSum((xbed[s][h] * cbed[s][h]) for (s,h) in sh) +
        pulp.lpSum((xstaff[s][h][b]*cstaff[b]) for (s,h,b) in shb)
    )
    
    prob = add_deterministic_constraints(
        xbed=xbed, 
        xstaff=xstaff, 
        UBbed=UBbed, 
        UBstaff=UBstaff,
        D=D, 
        K=K,
        R=R,
        sh=sh,
        shb=shb,
        prob=prob,
    )
    prob.solve(pulp.PULP_CBC_CMD(maxSeconds=60,fracGap=0.01))
    
    return prob


In [8]:
"""
These values can be altered to the specific requirements of the user
"""
specialties = list(itertools.chain(range(0, 29))) #Creates List of specialties
bands = list(itertools.chain(range(0,3))) #Creates lists of nursing bands

#Determining which hospitals are assigned to each region
region1 = [0,1,2]
region2 = [3,4]
region3 = [5]
region4 = [6]
region5 = [7,8,9]
region6 = [10]
hospitals = region1 + region2 +region3 + region4 +region5 +region6

regions = list(itertools.chain(range(0,6))) #Creats List of regions

D = [
    [2.1081,0.0000,0.0000,0.0000,9.1846,0.0000],
    [4.6079,0.0000,0.0000,0.0000,0.7734,0.0000],
    [16.0947,0.0000,0.0000,0.0000,9.8809,0.0002],
    [94.5387,57.7380,0.7489,8.7416,46.4786,0.0000],
    [0.0000,6.9952,0.0000,0.3121,12.9756,0.0000],
    [2.4192,0.0000,0.0000,0.0000,0.0000,0.0000],
    [14.5635,21.2838,0.0000,0.0000,17.2387,0.0000],
    [3.2480,0.0041,0.0000,0.0000,0.0000,0.0000],
    [13.0208,0.6065,0.0000,0.0000,20.1331,0.0725],
    [84.9712,0.9846,0.0115,0.0000,14.1695,0.0000],
    [46.5808,0.5390,0.0000,0.0000,21.8943,0.0006],
    [0.0000,9.8734,0.0000,0.0000,15.2880,0.0000],
    [2.1194,0.1222,0.0000,0.0000,1.1716,0.0002],
    [3.0718,0.0320,0.0000,0.0000,1.8339,0.0002],
    [7.1817,0.0000,0.0000,0.0000,0.0000,0.0000],
    [0.0000,0.0000,0.3426,0.3248,0.0000,0.0000],
    [1.1831,0.0000,0.0000,0.0000,0.0243,0.0000],
    [1.5828,0.0000,0.0000,0.0000,0.0000,0.0000],
    [2.5538,0.0028,0.0000,0.0000,0.1968,0.5232],
    [0.0586,0.0055,0.0000,0.0037,0.0131,0.0000],
    [0.0000,0.0000,0.0000,0.0000,0.0336,0.0002],
    [0.0146,0.0000,0.0000,0.0000,0.0026,0.0000],
    [0.2265,0.0000,0.0000,0.0000,0.0000,0.0000],
    [62.8610,32.9653,68.9710,31.9917,24.4659,0.0000],
    [29.8010,0.0000,0.0000,0.0000,27.8372,0.0000],
    [0.0001,0.0000,0.0000,0.0000,0.0000,0.0000],
    [0.0004,0.0015,0.0000,0.0000,0.0128,0.0000],
    [60.3126,0.6665,0.0000,0.0000,41.5186,0.0000],
    [12.3397,0.0521,0.0000,0.0000,0.2034,0.0702]
    ]

K = [
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,309,0,0,80],
    [588,0,75,163,0,80,73,309,26,0,0],
    [0,0,0,163,0,0,73,0,26,0,0],
    [588,28,75,163,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,0],
    [588,0,0,163,0,0,0,0,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,80],
    [588,0,0,163,0,80,0,309,0,0,0],
    [588,0,0,163,0,0,0,309,26,0,80],
    [0,0,0,0,12,0,0,0,26,16,80],
    [588,0,0,163,0,0,0,309,0,0,80],
    [588,28,0,163,0,0,0,309,0,0,80],
    [588,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,80,73,0,0,0,0],
    [588,28,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,80],
    [588,0,0,163,0,0,73,309,0,0,0],
    [0,0,0,0,0,0,0,309,0,0,80],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,0,0,0,0,0,0,0,80],
    [588,28,75,163,0,80,73,309,26,0,0],
    [588,0,0,0,0,0,0,309,0,0,0],
    [588,0,0,163,0,0,0,0,0,0,0],
    [588,0,0,163,0,0,0,309,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,0],
    [588,28,0,163,0,0,0,309,0,0,80]
    ]
R = [
    [0.250,0.250,0.250],
    [0.250,0.250,0.250],
    [0.125,0.125,0.125],
    [0.1,0.1,0.1],
    [0.1,0.1,0.1],
    [0.1,0.1,0.1],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.25,0.25,0.25],
    [0.25,0.25,0.25],
    [0.1,0.1,0.1],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.1,0.1,0.1],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.1,0.1,0.1],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.125,0.125,0.125],
    [0.25,0.25,0.25],
    [0.125,0.125,0.125],
    ] 

cbed = [
    [345,0,0,0,0,0,0,149,0,0,0],
    [526,0,0,0,0,0,0,1516,0,0,0],
    [396,0,0,0,0,0,0,895,0,0,551],
    [457,0,755,493,0,542,472,743,577,0,0],
    [0,0,0,942,0,0,1100,0,1021,0,0],
    [1672,216,1232,2404,0,0,0,0,0,0,0],
    [296,0,0,1270,0,0,0,1497,0,0,0],
    [481,0,0,501,0,0,0,0,0,0,0],
    [448,402,0,639,0,0,0,959,0,0,832],
    [390,0,0,94,0,65,0,611,0,0,0],
    [472,0,0,304,0,0,0,539,541,0,849],
    [0,0,0,0,172,0,0,0,325,443,360],
    [1007,0,0,528,0,0,0,292,0,0,241],
    [1299,1277,0,1218,0,0,0,1070,0,0,1176],
    [711,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,39,197,0,0,0,0],
    [2026,1940,0,0,0,0,0,264,0,0,0],
    [1273,0,0,0,0,0,0,0,0,0,0],
    [957,0,0,369,0,0,0,1416,0,0,174],
    [124,0,0,111,0,0,134,143,0,0,0],
    [0,0,0,0,0,0,0,1308,0,0,496],
    [945,0,0,0,0,0,0,1097,0,0,0],
    [633,0,0,0,0,0,0,0,0,0,1545],
    [1973,1274,972,975,0,1021,1983,1987,1455,0,0],
    [330,0,0,0,0,0,0,566,0,0,0],
    [141,0,0,139,0,0,0,0,0,0,0],
    [631,0,0,621,0,0,0,536,0,0,0],
    [678,651,0,610,0,0,0,873,0,0,0],
    [102,214,0,79,0,0,0,378,0,0,1122]
    ]

cstaff = [338.88,419.52,516.96]
UBbed = [588, 28, 75, 163, 12, 80,73,309,26,16,80]
UBstaff = [400,400,400]

In [9]:
prob = solve_deterministic_minimisation_problem(
    specialties,
    bands,
    region1,
    region2,
    region3,
    region4,
    region5,
    region6,
    hospitals,
    regions,
    D,
    K,
    R,
    cbed,
    cstaff,
    UBbed,
    UBstaff,
)

In [10]:
print("Solution Status = ", pulp.LpStatus[prob.status])
print("Total price = ", pulp.value(prob.objective))  
for v in prob.variables():
    if v.varValue >= 0:
        #print(v.varValue)
        print(v.name, "=", v.varValue)
production = [v.varValue for v in prob.variables()]

Solution Status =  Optimal
Total price =  1011291.5200000004
Xbed_0_0 = 3.0
Xbed_0_1 = 0.0
Xbed_0_10 = 0.0
Xbed_0_2 = 0.0
Xbed_0_3 = 0.0
Xbed_0_4 = 0.0
Xbed_0_5 = 0.0
Xbed_0_6 = 0.0
Xbed_0_7 = 10.0
Xbed_0_8 = 0.0
Xbed_0_9 = 0.0
Xbed_10_0 = 47.0
Xbed_10_1 = 0.0
Xbed_10_10 = 1.0
Xbed_10_2 = 0.0
Xbed_10_3 = 1.0
Xbed_10_4 = 0.0
Xbed_10_5 = 0.0
Xbed_10_6 = 0.0
Xbed_10_7 = 22.0
Xbed_10_8 = 0.0
Xbed_10_9 = 0.0
Xbed_11_0 = 0.0
Xbed_11_1 = 0.0
Xbed_11_10 = 0.0
Xbed_11_2 = 0.0
Xbed_11_3 = 0.0
Xbed_11_4 = 10.0
Xbed_11_5 = 0.0
Xbed_11_6 = 0.0
Xbed_11_7 = 0.0
Xbed_11_8 = 0.0
Xbed_11_9 = 16.0
Xbed_12_0 = 3.0
Xbed_12_1 = 0.0
Xbed_12_10 = 1.0
Xbed_12_2 = 0.0
Xbed_12_3 = 1.0
Xbed_12_4 = 0.0
Xbed_12_5 = 0.0
Xbed_12_6 = 0.0
Xbed_12_7 = 2.0
Xbed_12_8 = 0.0
Xbed_12_9 = 0.0
Xbed_13_0 = 4.0
Xbed_13_1 = 0.0
Xbed_13_10 = 1.0
Xbed_13_2 = 0.0
Xbed_13_3 = 1.0
Xbed_13_4 = 0.0
Xbed_13_5 = 0.0
Xbed_13_6 = 0.0
Xbed_13_7 = 2.0
Xbed_13_8 = 0.0
Xbed_13_9 = 0.0
Xbed_14_0 = 8.0
Xbed_14_1 = 0.0
Xbed_14_10 = 0.0
Xbed_14_2 =

Xstaff_1_4_2 = 0.0
Xstaff_1_5_0 = 0.0
Xstaff_1_5_1 = 0.0
Xstaff_1_5_2 = 0.0
Xstaff_1_6_0 = 0.0
Xstaff_1_6_1 = 0.0
Xstaff_1_6_2 = 0.0
Xstaff_1_7_0 = 1.0
Xstaff_1_7_1 = 1.0
Xstaff_1_7_2 = 1.0
Xstaff_1_8_0 = 0.0
Xstaff_1_8_1 = 0.0
Xstaff_1_8_2 = 0.0
Xstaff_1_9_0 = 0.0
Xstaff_1_9_1 = 0.0
Xstaff_1_9_2 = 0.0
Xstaff_20_0_0 = 0.0
Xstaff_20_0_1 = 0.0
Xstaff_20_0_2 = 0.0
Xstaff_20_10_0 = 1.0
Xstaff_20_10_1 = 1.0
Xstaff_20_10_2 = 1.0
Xstaff_20_1_0 = 0.0
Xstaff_20_1_1 = 0.0
Xstaff_20_1_2 = 0.0
Xstaff_20_2_0 = 0.0
Xstaff_20_2_1 = 0.0
Xstaff_20_2_2 = 0.0
Xstaff_20_3_0 = 0.0
Xstaff_20_3_1 = 0.0
Xstaff_20_3_2 = 0.0
Xstaff_20_4_0 = 0.0
Xstaff_20_4_1 = 0.0
Xstaff_20_4_2 = 0.0
Xstaff_20_5_0 = 0.0
Xstaff_20_5_1 = 0.0
Xstaff_20_5_2 = 0.0
Xstaff_20_6_0 = 0.0
Xstaff_20_6_1 = 0.0
Xstaff_20_6_2 = 0.0
Xstaff_20_7_0 = 1.0
Xstaff_20_7_1 = 1.0
Xstaff_20_7_2 = 1.0
Xstaff_20_8_0 = 0.0
Xstaff_20_8_1 = 0.0
Xstaff_20_8_2 = 0.0
Xstaff_20_9_0 = 0.0
Xstaff_20_9_1 = 0.0
Xstaff_20_9_2 = 0.0
Xstaff_21_0_0 = 1.0
Xstaff_21_0_1